
#  RAG and Python function Deployment

In this notebook, we create and deploy a python function that takes a user defined question as input and generates an answer using the RAG process. The function does the following:
- Connects to Elasticsearch or Milvus Vector Database and retrieves the top N results (over a certain threshold) matching the question from the vector index.
- Includes the question, vector search results into an optimized prompt template for the LLM
- Runs LLM inference on watsonx.ai to generate an answer
- Checks for hallucinations using word overlap or sentence transformer algorithms and threshold values.
- Updates the feedback provided by the user based on the LLM result in log records. 


**Note**: It is recommended to run this notebook in a Python environment on CPD software with a GPU-enabled or high vCPU and RAM hardware configuration, as generating embeddings may require significant memory.
## Contents

This notebook contains the following parts:

- [Pre-Requisite Libraries and Dependencies](#Setup)
- [Import Parameter Set, Credentials and Helper function script](#parameterimport)
- [Initialise Deployment Space](#DeploymentSpace)
- [Promote Assets to Deployment Space](#promote)
- [Create Scoring Function](#ScoringFunction)
- [Deploy Scoring Function in Deployment Space](#DeployScoringFunction)
- [Test the scoring function](#scoring)
- [Update the feedback log](#feedback-logging)
- [Update parameter set in the project & deployment space](#updateParameters)




<a id="Setup"></a>
### Pre-Requisite Libraries and Dependencies

Download and import mandatory libraries and dependencies

In [ ]:
!pip install --upgrade elasticsearch | tail -n 1
!pip install ibm_watsonx_ai==1.1.6 | tail -n 1
!pip install pymilvus | tail -n 1

Restart the kernel, after performing pip install.

In [ ]:
import os
import json
import string
import time
import uuid
from ibm_watsonx_ai import APIClient,Credentials
from ibm_watsonx_ai import __version__

import warnings
warnings.filterwarnings("ignore")

from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats


from pymilvus import(connections,FieldSchema,DataType,Collection,CollectionSchema,utility)

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()

<a id="parameterimport"></a>
### Import Parameter Sets, credentials and Helper functions script.

The below cells imports the parameter set, credentials for watsonx.ai and helper function script. 

In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]
parameters={}
for parameter_set in parameter_sets:
    parameter_vals= wslib.assets.get_asset(parameter_set, "parameter_set", raw=True)
    if 'entity' in parameter_vals and 'parameter_set' in parameter_vals['entity'] and 'parameters' in parameter_vals['entity']['parameter_set']:
        parameter_vals = parameter_vals['entity']
    params = {param['name']: param['value'] for param in parameter_vals['parameter_set']['parameters']}
    parameters.update(params)    

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
space_uid = parameters['watsonx_ai_space_id']

if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=WML_SERVICE_URL)
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials=Credentials(token=os.environ['USER_ACCESS_TOKEN'],url=hostname,instance_id='openshift')

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)

### Import Prompt Template

Imports the prompt template based on the parameter set by the user. 

In [ ]:

prompt_mgr = PromptTemplateManager(
                credentials=wml_credentials,
                project_id=project_id
                )


df_prompt = prompt_mgr.list()
prompt_template_id=df_prompt.loc[df_prompt['NAME'] == parameters['llm_prompt_template_file'], 'ID'].values[0]
print("Currently using the ", parameters['llm_prompt_template_file'], "prompt template.")

### Create Connection and Promote to Space

Retrieves the connection information from the connection asset you created if the asset exists. \
Connects to the vector database depending on whether its an elasticsearch or milvus connection and initilizes a client respectively. Promotes the connection id to the deployment space. 

In [ ]:
connection_name=parameters["connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    db_connection = wslib.get_connection(connection_name)
    
    connection_datatypesource_id=client.connections.get_details(db_connection['.']['asset_id'])['entity']['datasource_type']
    connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",connection_type)
    if connection_type=="elasticsearch":
        es_client=rag_helper_functions.create_elastic_client(db_connection)
    elif connection_type=="milvus":
        db_connection['database']=db_connection['database'] if 'database' in db_connection else 'default'
        connection_params = {
            'alias': db_connection['database'],
            'host': db_connection['host'],
            'port': db_connection['port'],
            'user': db_connection['username'],
            'password': db_connection['password'],
            'secure': True
        }
        if 'ssl_certificate' in db_connection :
            ssl_certificate_content = db_connection.get('ssl_certificate') if db_connection.get('ssl_certificate') else ""
            cert_file_path = 'milvus_conn.cert'
            with open(cert_file_path, 'w') as file:
                file.write(ssl_certificate_content)
            connection_params['server_pem_path'] = cert_file_path
        milvus_client = connections.connect(**connection_params)
        print("Successfully connected to milvus database")
        
    connection_id = client.spaces.promote(db_connection['.']['asset_id'], project_id, space_uid)
    print("Promoted the connection to deployment space")
else:
    db_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.")
    

The notebook, by default, will look for a log connection asset in the project named milvus_connect or elasticsearch_connect. You can set this up by following the instructions in the project readme. This code checks if a specified connection exists in the project. If found, it retrieves the connection details and identifies the connection type. This can either be Elasticsearch or Milvus. \
Depending on the connection type, it establishes a connection to the appropriate database. If the connection is not found, it raises an error indicating the absence of the specified connection in the project.

In [ ]:
log_connection_name=parameters["log_connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == log_connection_name), None)):
    print(log_connection_name, "Log connection found in the project")
    log_db_connection = wslib.get_connection(log_connection_name)
    print("Successfully retrieved the log connection details")
    
    log_connection_datatypesource_id=client.connections.get_details(log_db_connection['.']['asset_id'])['entity']['datasource_type']
    log_connection_type = client.connections.get_datasource_type_details_by_id(log_connection_datatypesource_id)['entity']['name']
    
    log_client = None
    
    print("Log Connection type is identified as:",log_connection_name)
    if log_connection_type=="elasticsearch":
        log_client=rag_helper_functions.create_elastic_client(log_db_connection)
    elif log_connection_type=="milvus":
        log_db_connection['database']=log_db_connection['database'] if 'database' in log_db_connection else 'default'
        log_connection_params = {
            'alias': log_db_connection['database'],
            'host': log_db_connection['host'],
            'port': log_db_connection['port'],
            'user': log_db_connection['username'],
            'password': log_db_connection['password'],
            'secure': True
        }
        
        if 'ssl_certificate' in log_db_connection :
            log_ssl_certificate_content = log_db_connection.get('ssl_certificate') if log_db_connection.get('ssl_certificate') else ""
            log_cert_file_path = 'milvus_conn.cert'
            with open(log_cert_file_path, 'w') as file:
                file.write(log_ssl_certificate_content)
            log_connection_params['server_pem_path'] = log_cert_file_path
        log_client = connections.connect(**log_connection_params)
        print("Successfully connected to milvus database")
        
    log_connection_id = client.spaces.promote(log_db_connection['.']['asset_id'], project_id, space_uid)
    print("Promoted the connection to deployment space")

    
else:
    log_db_connection=""
    log_connection_id = ""
    log_connection_type=""
    print(f"No log connection named {log_connection_name} found in the project. Remove log connection name from parameter set to disable logging." \
          if not log_connection_name =='' else f"No log connection specified. Logging will be disabled.")
    

 
<a id="DeploymentSpace"></a>
### Get the Deployment Space details and set default space.

In the next steps we save and deploy the pipeline. The pipeline can be saved and deployed in the same way we save and deploy models.

Before we deploy a function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space.

**Creating a Deployment Space** <br>
Before we save the function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their functions or models. We can save models, functions and data assets in this space.
If you do not have space already created, you can use Deployment Spaces Dashboard to create one. Follow the steps : 
* Navigate to Deployments
* Click New Deployment Space
* Enter Deployment space name, for eg : **'RAG Deployment Space'**

**In case of watsonx as a service**
* Follow the steps to create the space same as above. 
* Select Cloud Object Storage
* Select Watson Machine Learning instance and press Create
* `Under Manage project` > `Services & Integrations` ensure that your WML service that you provisioned is associated there. 


The steps involved in saving and deploying the pipeline are detailed in the following cells. We will use the `ibm-watson-machine-learning` package to complete these steps.

**Setting space id in parameters set**
* In your Deployment Space, copy the space ID by going to **Manage >** copy the Space GUID
* In your Project, open the `RAG_parameter_set` found in **Data Asset > Configuration**.
* Edit the parameter name `watsonx_ai_space_id` and paste the guid here.

In [ ]:
space = client.set.default_space(space_uid)
if space != 'SUCCESS':
    print("Please ensure that the space exists & try again.")
else:
    print("Default WML Client setting : ", space)

<a id="promote"></a>
### Promote Necessary Assets to the Deployment Space.

The following assets are promoted to the deployment space to be used in the deployed function
* Elasticsearch / Milvus Connection
* Parameter sets
* LLM Prompt template 
* RAG Helper functions python script

Promote the assets and the prompt template to the space to be used by the deployed function.

In [ ]:
promote_files_to_space=['rag_helper_functions.py']
space_asset_dict={}
for file in wslib.assets.list_assets('data_asset'):
    if file['name'] in promote_files_to_space:
        space_asset_dict[file['name']] = client.spaces.promote(file['asset_id'], project_id, space_uid)
        print(file['name'],"promoted to deployment space")

for parms in wslib.assets.list_assets('parameter_set'):
    if parms['name'] in parameter_sets:
        space_asset_dict[parms['name']] = client.spaces.promote(parms['asset_id'], project_id, space_uid)
        print(parms['name'],"promoted to deployment space")

space_asset_dict       

In [ ]:
print("Promoting the prompt template to deployment space")
client.spaces.promote(prompt_template_id, project_id, space_uid)

Asset details of all data assets promoted to the space to be used by the deployed function

In [ ]:
# common ai parameters passed to deployed function
ai_params = {'space_id': space_uid, 'space_asset_dict': space_asset_dict, 'environment': environment, 
             'connection_name': connection_name, 'connection_id': connection_id,
             "connection_type":connection_type,"project_id":parameters.get('watsonx_ai_project_id') or project_id,
             'log_connection_id': log_connection_id, 'log_connection_type': log_connection_type,
             'log_pii_removal': ('log_pii_removal' in parameters and parameters['log_pii_removal'].lower() == 'true'),'wml_credentials':wml_credentials.to_dict() }

### Create Software Specifications
Below code snippet performs the following actions:

It defines various configuration constants for setting up a Python runtime environment, including the base software specification, custom software specification name, package extension name, and a configuration file path. It creates a configuration content string in YAML format that specifies the dependencies needed. The configuration content is then written to a file named `config.yaml`. Metadata properties are prepared for storing the package extension using the configuration file, and these properties are saved to the IBM Watson Machine Learning client. The unique identifier (UID) for the stored package extension is retrieved. Metadata properties are also prepared for a new software specification that uses the base specification, and this is stored to the Watson Machine Learning client. The unique identifier (UID) for the newly created software specification is retrieved. Finally, the package extension is associated with the new software specification by adding the package extension UID to it.

The respective software specifications uid as a result is provided as a parameter during the deployment of function, so that pipeline function will have the dependent libraries for execution on the Deployment Space at runtime.

In [ ]:

BASE_SW_SPEC_NAME = "runtime-24.1-py3.11"
sw_spec_name = "rag_qna_sw_spec_"+str(uuid.uuid1()).split('-')[0]
pkg_ext_name = "rag_qna-py3.11"
CONFIG_PATH = "config.yaml"
CONFIG_TYPE = "conda_yml"
CONFIG_CONTENT = f"""
        name: python310
        channels:
          - empty
        dependencies:
          - pip:
            - langchain
            - setuptools
            - langchain_milvus
            - sentence_transformers
            - langchain-core
            - langchain_ibm
            - ibm-watsonx-ai=={__version__}
            - elasticsearch
            - langchain-community
            - pymilvus
            - langchain_chroma
            - langchain_elasticsearch
        prefix: /opt/anaconda3/envs/python310
"""
with open(CONFIG_PATH, 'w', encoding='utf-8') as f:
    f.write(CONFIG_CONTENT)
pkg_extn_meta_props = {
    client.package_extensions.ConfigurationMetaNames.NAME: pkg_ext_name,
    client.package_extensions.ConfigurationMetaNames.TYPE: CONFIG_TYPE
}

pkg_extn_details = client.package_extensions.store(meta_props=pkg_extn_meta_props, file_path=CONFIG_PATH)
pkg_extn_uid = client.package_extensions.get_id(pkg_extn_details)

sw_spec_meta_props = {
    client.software_specifications.ConfigurationMetaNames.NAME: sw_spec_name,
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        'guid': client.software_specifications.get_id_by_name(BASE_SW_SPEC_NAME)
    }
}

try:
    sw_spec_details = client.software_specifications.store(meta_props=sw_spec_meta_props)
    sw_spec_id = client.software_specifications.get_id(sw_spec_details)

    client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_uid)
except Exception as e:
    print(f"An error occurred: {e}")
    print("\nExisting software_specification will be used")
    sw_spec_id=client.software_specifications.get_id_by_name(sw_spec_name)
    client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_uid)
    
import os
os.remove(CONFIG_PATH)

<a id="ScoringFunction"></a>

### Python scoring pipeline function

Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to perform the list of tasks below and pass it to the pipeline for getting responses from the LLM, analysing and logging the response all within the deployed function.
The function handles the following tasks:

1. Initializing the WML client.
2. Downloading assets promoted to the deployment space.
3. Calling the Python helper functions script.
4. Initializing the Vector Database client and creating a vector store for Elasticsearch or Milvus.
5. Initializing LLM with parameters.
6. Creating an LLM chain using the LLM, vector store, and prompt template.
7. Retrieving a response from the LLM using LLM Chain and checking for hallucinations.
8. Writing response data to a log record in Elasticsearch or Milvus and creating an index/collection for logs if it doesn't exist. For this purpose it follows below steps:
   - Make connection to vector store specified in log connection type, i.e, either milvus or elastic search. 
   - Depending on the connection type
     - If connection type is 'Milvus', it uses pymilvus to setup connection. An embedding model specified in the parameters is initialized for generating embeddings, and index parameters are defined for indexing with specific metrics and settings. If collection is not present, then collection with specific field/document is created to store log records. 
     - If connection type is 'Elasticsearch', it uses langchain_elasticsearch to setup connection. If index is not present, then the index is created to store log records. \
   - Returning the result to the user.
   - Providing the document ID of the log record.
   - Updating the log record with the user's feedback (through a separate function call).

In [ ]:
def rag_scoring_pipeline_function(params=ai_params):
    #import subprocess
    #subprocess.check_output("pip install --upgrade elasticsearch --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install --upgrade sentence_transformers --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-community --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-core --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-elasticsearch --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install ibm-watsonx-ai --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_ibm --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install pymilvus --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_milvus --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_chroma --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install pydantic==1.10.11 --user", stderr=subprocess.STDOUT, shell=True)

    from langchain_ibm import WatsonxLLM
    import requests
    import os
    import json
    import warnings
    import time
    import hashlib
    from datetime import datetime
    import re
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd
    from ibm_watsonx_ai import APIClient
    from ibm_watsonx_ai.foundation_models import Embeddings
    from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
    from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
    from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watsonx_ai.foundation_models import Model
    from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
    from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats, DecodingMethods
    from ibm_watsonx_ai import Credentials
    from ibm_watson_studio_lib import access_project_or_space
    from sentence_transformers import SentenceTransformer, util
    import string
    from langchain.chains import LLMChain
    # from ibm_watsonx_ai.foundation_models.extensions.rag import VectorStore

    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.runnables import RunnableParallel

    from pymilvus import (connections, FieldSchema, DataType, Collection, CollectionSchema, utility)

    s = requests.Session()

    # Conditional import, since watson_nlp might not be available in runtime environment
    watson_nlp_available = False
    if params['log_pii_removal']:
        try:
            import watson_nlp
            watson_nlp_available = True
        except ImportError:
            pass

    # from langchain.globals import set_debug
    # set_debug(True)

    # AI Params passsed to function
    wml_credentials= params['wml_credentials']

    hostname = wml_credentials.get('url') # url 'wml_credentials'
  
    ibm_api_key = wml_credentials.get('apikey','')  # ibm cloud api_key
    environment = params['environment']  # cloud/on-premise
    token = wml_credentials.get('token','') # wml token
    space_asset_dict = params['space_asset_dict']
    
    inference_project_id = params.get('project_id') # RAG project id



    def validate_answer_against_sources(response_answer, source_documents, similarity_threshold=0.5):
        model = SentenceTransformer('all-MiniLM-L6-v2')
        source_texts = [doc.page_content for doc in source_documents]
        answer_embedding = model.encode(response_answer, convert_to_tensor=True)
        source_embeddings = model.encode(source_texts, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)
        print(cosine_scores[0])
        if any(score.item() > similarity_threshold for score in cosine_scores[0]):
            matching_scores = [score.item() for score in cosine_scores[0] if score.item() > similarity_threshold]
            confidence_score = max(matching_scores)
            return {'isHallucination': False, 'confidence_score': round(float(confidence_score), 2)}
        else:
            confidence_score = sum(cosine_scores[0]) / len(source_documents)
            print(confidence_score)
            return {'isHallucination': True, 'confidence_score': round(float(confidence_score), 2)}

    def is_hallucination(response_answer, source_documents, threshold_overlap_max=0.3,
                         threshold_overlap_score_concat=0.4):
        stop_words = {
            'a', 'about', 'after', 'all', 'also', 'am', 'an', 'and', 'another', 'any', 'are', 'as', 'at',
            'be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by',
            'came', 'can', 'come', 'could', 'did', 'do', 'each', 'for', 'from', 'get', 'got',
            'has', 'had', 'he', 'have', 'her', 'here', 'him', 'himself', 'his', 'how',
            'i', 'if', 'in', 'into', 'is', 'it', 'like',
            'make', 'many', 'me', 'might', 'more', 'most', 'much', 'must', 'my', 'never', 'now',
            'of', 'on', 'only', 'or', 'other', 'our', 'out', 'over',
            'said', 'same', 'should', 'since', 'some', 'still', 'such',
            'take', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'these', 'they', 'this',
            'those', 'through', 'to', 'too', 'under', 'up', 'very', 'was', 'way', 'we', 'well', 'were',
            'what', 'where', 'which', 'while', 'who', 'with', 'would', 'you', 'your'
        }

        regex = re.compile(r"\b\w+(?:['-_]\w+)?\b")

        def calculate_textual_overlap(text1, text2):
            # tokenize
            text1_tokens = regex.findall(text1.lower())
            text2_tokens = regex.findall(text2.lower())

            # remove stop words
            text1_tokens = [t for t in text1_tokens if t not in stop_words]
            text2_tokens = [t for t in text2_tokens if t not in stop_words]

            # compute overlap
            if len(text1_tokens) > 0:
                text1_tokens = set(text1_tokens)
                text2_tokens = set(text2_tokens)
                return len(text1_tokens.intersection(text2_tokens)) / len(text1_tokens)
            else:
                return 0

        def overlap_score_concat(generated_text, passages):
            passages_text = ' '.join(passages)
            return calculate_textual_overlap(generated_text, passages_text)

        def overlap_score_max(generated_text, passages):
            return max([calculate_textual_overlap(generated_text, passage_text) for passage_text in passages])

        # Get the LLM response along with source document text
        llm_response_text = response_answer
        source_docs_passages = [doc.page_content for doc in source_documents]

        result_overlap_score_max = 0.0
        result_overlap_score_concat = 0.0

        if len(source_docs_passages):

            # Call overlap score calculations
            result_overlap_score_max = overlap_score_max(llm_response_text, source_docs_passages)
            result_overlap_score_concat = overlap_score_concat(llm_response_text, source_docs_passages)

            print('result_overlap_score_max: ' + str(result_overlap_score_max))
            print('result_overlap_score_concat: ' + str(result_overlap_score_concat))

            # Define thresholds (these should be put in a parameter set or template)
            # Need to experiment with what values to use

            response = {'isHallucination': False,
                        'text': "",
                        'maxOverlapScore': round(result_overlap_score_max, 2),
                        'concatOverlapScore': round(result_overlap_score_concat, 2)}

            # Decide if hallucination or not
            if result_overlap_score_max > threshold_overlap_max and result_overlap_score_concat > threshold_overlap_score_concat:

                response['isHallucination'] = False
            else:
                print("Hallucination")
                response['isHallucination'] = True
                response['text'] = "Sorry, I can not generate a valid answer to your question."
        else:
            response = {'isHallucination': True,
                        'text': "Sorry, I can not find an answer to your question in the available documents.",
                        'maxOverlapScore': round(result_overlap_score_max, 2),
                        'concatOverlapScore': round(result_overlap_score_concat, 2)}

        return response

    try:
        client = APIClient(wml_credentials)
        client.set.default_space(params['space_id'])
    except Exception as e:

        print(f"Error initializing WML client: {str(e)}")

        raise

    try:
        helper_function_name = 'rag_helper_functions.py'
        helper_function_path = client.data_assets.download(space_asset_dict[helper_function_name], helper_function_name)
        script_name = os.path.basename(helper_function_path).replace('.py', '')
        rag_helper_functions = __import__(script_name)
        space_parameter_dict = {key: value for key, value in space_asset_dict.items() if "parameter" in key.lower()}

        parameters_list = []
        for i in space_parameter_dict.values():
            parameters_list = parameters_list + client.parameter_sets.get_details(i)['entity']['parameter_set']['parameters']
        parameters = {param['name']: param['value'] for param in parameters_list}

        hallucination_threshold_max_text_overlap = parameters['hallucination_threshold_max_text_overlap']
        hallucination_threshold_concatenated_text_overlap = parameters['hallucination_threshold_concatenated_text_overlap']
        #print(client.credentials.url)
        #credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=client.credentials.url)


    except Exception as e:

        print(f"Error initializing the parameters: {str(e)}")
        raise

    index_name = parameters["vector_store_index_name"]
    model_id = parameters['elastic_search_model_id']
    # connection_type=params['connection_type']

    try:
        db_connection = client.connections.get_details(params['connection_id'])['entity']['properties']
        
        if params['connection_type'] == "elasticsearch":
            es_client = rag_helper_functions.create_elastic_client(db_connection)

            from langchain_elasticsearch import ElasticsearchStore
            vector_store = ElasticsearchStore(
                es_connection=es_client,
                index_name=parameters["vector_store_index_name"],
                strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(
                    model_id=parameters['elastic_search_model_id'])
            )



        ############################################# MILVUS #########################################################
        elif params['connection_type'] == "milvus":
            from langchain_milvus import Milvus
            db_connection['database'] = db_connection.get('database', 'default')
            milvus_credentials = {'database': db_connection['database'], 'password': db_connection['password'],
                                  'port': db_connection['port'], 'host': db_connection['host'], "secure": True,
                                  'user': db_connection['username'] }
            if 'ssl_certificate' in db_connection :
                ssl_certificate_content = db_connection.get('ssl_certificate') if db_connection.get('ssl_certificate') else ""
                cert_file_path = 'milvus_conn.cert'
                with open(cert_file_path, 'w') as file:
                    file.write(ssl_certificate_content)
                milvus_credentials['server_pem_path']=cert_file_path
            
            
            print("using the model", parameters['embedding_model_id'], "to create embeddings")
            
            if environment=="cloud":
                embedding = Embeddings(
                model_id=parameters['embedding_model_id'],
                credentials=wml_credentials,
                project_id=inference_project_id,
                verify=True
                )
            elif environment=="on-prem":
                print("Downloading the embedding model from HuggingFace")
                from langchain_community.embeddings import HuggingFaceEmbeddings
                embedding=HuggingFaceEmbeddings(model_name=parameters['embedding_model_id'])

            index_params = {"index_type": "HNSW","metric_type": "L2","params":{"M": 16,"efConstruction": 200,"efSearch": 16}}

            vector_store = Milvus(
                embedding_function=embedding,
                connection_args=milvus_credentials,
                index_params=index_params,
                primary_field='id',
                # text_field="page_content",
                collection_name=parameters["vector_store_index_name"]

            )
            print('vector store: ', vector_store)
            print("Milvus Vector Store Created")
    ############################################# MILVUS #########################################################

    except Exception as e:

        print(f"Error creating connection and vector store: {str(e)}")

    try:

        # Gen AI via WatsonX.AI

        if environment == "cloud":
            print("Running the prompt on watsonx.ai cloud ..")
            model_credentials = wml_credentials
            
        else:
            if parameters['watsonx_ai_project_id']:
                print("Running the prompt on watsonx.ai cloud ..")
                model_credentials = {'url': parameters['watsonx_ai_url'],
                                      'api_key': parameters['watsonx_ai_api_key'],
                                      'project_id': inference_project_id}
                
            else:
                print("Running the prompt on watsonx.ai on-prem ..")
                model_credentials = wml_credentials
                



       # if environment == "cloud":
       #     prompt_credentials = model_credentials
       # elif environment == "on-prem":
       #     prompt_credentials = client.wml_credentials

        print("Reading Prompt")
        prompt_mgr = PromptTemplateManager(
            credentials=wml_credentials,
            space_id=params['space_id'])

        df_prompt = prompt_mgr.list()
        print(parameters['llm_prompt_template_file'])
        prompt_template_id = df_prompt.loc[df_prompt['NAME'] == parameters['llm_prompt_template_file'], 'ID'].values[0]
        prompt_model = prompt_mgr.load_prompt(prompt_template_id)

        if 'stop_sequences' in prompt_model.model_params:
            prompt_model.model_params['stop_sequences'] = ['<|endoftext|>']
            
        model = Model(
            model_id=prompt_model.model_id,
            params=prompt_model.model_params,
            credentials=model_credentials,
            project_id=inference_project_id
        )

        print("model initialized")
        QA_CHAIN_PROMPT = prompt_mgr.load_prompt(prompt_template_id, PromptTemplateFormats.LANGCHAIN)

        # retriever = vector_store.as_langchain_retreiver(search_type="similarity_score_threshold",search_kwargs={"k":parameters['rag_es_top_n_results'],"score_threshold":float(parameters['rag_es_min_score'])})

        def retrieve_with_scores(query, filter=None):
            search_kwargs = {
                "k": parameters['vectorsearch_top_n_results'],
                "score_threshold": float(parameters['rag_es_min_score']),
                "include_scores": True,
                "verbose": True
            }
            if filter is not None:
                search_kwargs["filter"] = filter

            # results = vector_store.search(query, **search_kwargs)

            if params['connection_type'] == "elasticsearch":
                results = vector_store.similarity_search_with_relevance_scores(query, **search_kwargs)
            elif params['connection_type'] == "milvus":
                results = vector_store.similarity_search_with_score_by_vector(embedding.embed_query(query), **search_kwargs)
            return [{"document": doc, "score": score} for doc, score in results]

        def format_docs(docs_with_scores):
            return "".join([f"[Document]\n{d['document'].page_content}[End]\n\n" for d in docs_with_scores])

        print("retriever created")
        llm_chain = (
                RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
                | QA_CHAIN_PROMPT
                | model.to_langchain()
                | StrOutputParser()
        )
        print("llm chain created")

        def retrieve_with_scores_wrapper(inputs):
            query = inputs["query"]
            filter = inputs.get("filter")
            return retrieve_with_scores(query, filter)

        rag_chain_with_source = RunnableParallel({
            "context": retrieve_with_scores_wrapper,
            "question": RunnablePassthrough(),
        }).assign(answer=llm_chain)
        print("rag chain created")



    except Exception as e:

        print(f"Error creating LLM Chain: {str(e)}")

    ################# Logger Class ###########

    # Logger class for updating Elasticsearch log index
    class rag_logger:

        def __init__(self, parameters):
            self.log_client = None
            print("logging", parameters['log_index_name'])
            if not 'log_index_name' in parameters or parameters['log_index_name'] == '':
                self.logger_status = f"logging is deactivated"
            elif not 'log_connection_type' in params or params["log_connection_type"] == '':
                self.logger_status = f"logging is deactivated"
            else:
                self.log_index_name = parameters['log_index_name']
                self.field_names = []
                try:
                    self.log_connection = client.connections.get_details(params['log_connection_id'])['entity']['properties']
                except:
                    self.log_connection = {key[4:]: parameters[key] for key in parameters.keys() if key.startswith('log_')}
                try:
                    
                    self.log_connection_type = params["log_connection_type"]

                    print("Log Connection type is identified as:", self.log_connection_type)
                    if self.log_connection_type == "elasticsearch":
                        self.log_client = rag_helper_functions.create_elastic_client(self.log_connection)
                        idx_create_status = self.log_client.options(ignore_status=400).indices.create(index=self.log_index_name)
                        self.logger_status = f"logger: connection established, log index {self.log_index_name} {'created' if not 'status' in idx_create_status else 'already exists' if idx_create_status['status'] == 400 else 'DEFECTIVE'}"

                    elif self.log_connection_type == "milvus":
                        
                        self.log_connection['database']=self.log_connection.get('database','default')
                        self.log_connection_params = {
                            'database': self.log_connection['database'],
                            'host': self.log_connection['host'],
                            'port': self.log_connection['port'],
                            'user': self.log_connection['username'],
                            'password': self.log_connection['password'],
                            'secure': True
                        }
                        
                        if 'ssl_certificate' in self.log_connection :
                            self.log_ssl_certificate_content = self.log_connection.get('ssl_certificate') if self.log_connection.get('ssl_certificate') else ""
                            self.log_cert_file_path = 'log_conn.cert'
                            with open(self.log_cert_file_path, 'w') as file:
                                file.write(self.log_ssl_certificate_content)
                            self.log_connection_params['server_pem_path'] = self.log_cert_file_path
                        
                        
                        connections.connect(**self.log_connection_params)
                        self.log_client = self.log_connection_type
                        
                        # Create a collection
                        collection_name = parameters['log_index_name']
                        if collection_name not in utility.list_collections():

                            fields = [
                                FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, max_length=65535),
                                FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="response", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="source_documents", dtype=DataType.JSON),
                                FieldSchema(name="Hallucination_Detection", dtype=DataType.JSON),
                                FieldSchema(name="log_timestamp", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="feedback", dtype=DataType.JSON),
                                FieldSchema(name="elapsed_times", dtype=DataType.JSON),
                                FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1024)
                                # Ensure the embedding dimension is correct
                            ]

                            self.field_names = [field.name for field in fields]
                            

                            # Create the collection schema
                            schema = CollectionSchema(fields, description="Collection for document embeddings", enable_dynamic_field=True)

                            # Create the index for the embeddings field
                            index_params = {
                                "index_type": "HNSW",
                                "metric_type": "L2",
                                "params": {
                                    "M": 16,
                                    "efConstruction": 200,
                                    "efSearch": 16
                                }
                            }
                            self.collection = Collection(name=collection_name, schema=schema, dynamic_field=True)
                            self.collection.create_index(field_name="vector", index_params=index_params)
                            self.logger_status = f"logger: connection established, log index {self.log_index_name}", 'created.'
                        else:
                            self.collection = Collection(name=collection_name)
                            self.field_names = [field.name for field in self.collection.schema.fields]
                            self.logger_status = f"logger: connection established, log index {self.log_index_name}", 'already exists.'

                        self.collection.load()

                        print("Milvus Conection Created")
                except Exception as e:
                    self.logger_status = f"logger: ERROR: {str(e)}"
                    self.log_client = None

                # Personally identifiable information (PII) detection
                self.pii_detection_model = None
                pii_status = 'inactive'
                self.pii_model_syntax = None
                self.pii_model_bilstm = None
                self.pii_model_rbr = None
                if watson_nlp_available:
                    try:
                        self.pii_model_syntax = watson_nlp.load('syntax_izumo_en_stock')
                        # Bidirectional Long Short-Term Memory (BiLSTM) model for names detection
                        self.pii_model_bilstm = watson_nlp.load('entity-mentions_bilstm_en_pii')
                        # Rule-Based Reasoner (RBR) for credit / social numbers and email detection
                        self.pii_model_rbr = watson_nlp.load('entity-mentions_rbr_multi_pii')
                        pii_status = 'active'
                    except:
                        pii_status = 'model load failed'
                self.logger_status = f"{self.logger_status}, PII suppression status: {pii_status}"

        def status(self):
            return self.logger_status if self.logger_status else 'unknown'

        def remove_pii(self, text):
            # remove PII from given text
            if self.pii_model_bilstm and self.pii_model_rbr:
                for model_run in [lambda text: self.pii_model_bilstm.run(self.pii_model_syntax.run(text)), lambda text: self.pii_model_rbr.run(text, language_code='en')]:
                    pii_detection = model_run(text)
                    begin = 0
                    parts = []
                    for _pii in pii_detection.mentions:
                        parts.append(text[begin:_pii.span.begin])
                        begin = _pii.span.end
                    parts.append(text[begin:len(text)])
                    text = 'XXX'.join(parts)
            return text

        def generate_hash(self, content):
            return hashlib.sha256(content.encode()).hexdigest()

        def create_log_record(self, question, content, times, feedback):
            log_id = ''
            if self.log_client:
                # compile LOG record (RAG response + question + time meassurements)
                log_record = {**content, 'log_timestamp': datetime.now().isoformat()}
                if question:
                    log_record['question'] = self.remove_pii(question)  # remove PII from user's question ("Hi I am Joe ...")
                if times:
                    log_record['elapsed_times'] = {times[i][0]: "{:.3f}".format(times[i][1] - times[i - 1][1]) for i in range(1, len(times))}
                if feedback:
                    log_record['feedback'] = feedback
                if content and 'response' in content:
                    log_record['response'] = self.remove_pii(
                        log_record['response'])  # remove PII if genAI refers to user's question ("Hi Joe, ...")
                res = None
                if self.log_connection_type == 'elasticsearch':
                    res = self.log_client.index(index=self.log_index_name, document=log_record)
                elif self.log_connection_type == 'milvus':
                    log_id = self.generate_hash(str(datetime.now().timestamp()))
                    log_record.update({'id': log_id, 'vector': embedding.embed_documents([log_record['question']])[0],
                                       'Hallucination_Detection': log_record['Hallucination Detection'],
                                       'feedback': {}})
                    log_record.pop('Hallucination Detection')
                    res = self.collection.insert(data=log_record)
                    self.collection.flush()
                    print(f'inserted data: {res.primary_keys[0]}')
                if res != None:
                    if self.log_connection_type == 'elasticsearch':
                        log_id = res['_id']
                    elif self.log_connection_type == 'milvus':
                        log_id = res.primary_keys[0]
                    status = 'ok'
                else:
                    status = 'log cannot be written'

            else:
                status = self.logger_status

            return status, log_id

        def update_log_record(self, feedback):

            if self.log_client:
                if not 'log_id' in feedback:
                    status = "log id not provided"
                else:
                    log_id = feedback['log_id']
                    feedback.pop("log_id")
                    if len(feedback.keys()) < 1:
                        # no feedback values
                        status = "feedback not provided"
                    else:
                        try:
                            # update document
                            if self.log_connection_type == 'elasticsearch':
                                idx_upd_status = self.log_client.options(ignore_status=404).update(index=self.log_index_name, id=log_id, doc={'feedback': feedback})
                                status = "ok" if not 'status' in idx_upd_status else (f"log id {str(log_id)} not found" if idx_upd_status['status'] == 404 else f"update failed, status {str(idx_upd_status['status'])}")
                            elif self.log_connection_type == 'milvus':
                                
                                existing_record = self.collection.query(expr=f"id=='{log_id}'", output_fields=self.field_names)
                                if len(existing_record) >= 1:
                                    existing_record = existing_record.pop()
                                    existing_record.update({'feedback': feedback})
                                    res = self.collection.upsert(data=existing_record)
                                    status = 'ok'
                                else:
                                    status = f"log id {str(log_id)} not found"
                        except Exception as e:
                            status = f"update of index {self.log_index_name} failed."
            else:
                status = self.logger_status

            return status

    logger = rag_logger({**parameters, **params})
    print(f"log status : {logger.status()}")

    def score(payload):

        logging = True
        question = None
        query_filter = None
        status = f"invalid parameters"
        fields = {payload['input_data'][0]['fields'][i]: payload['input_data'][0]['values'][0][i] for i in range(min(len(payload['input_data'][0]['fields']), len(payload['input_data'][0]['values'][0])))}

        if '_log' in fields:
            logging = fields.pop('_log')

        if 'question' in fields:
            question = fields.pop('question')

        if '_content' in fields:
            print('calling insert')
            (status, log_id) = logger.create_log_record(None, fields['_content'], None, None) if logger else (f"log not available", '')
            return {"predictions": [{"fields": ["status", "log_id"], "values": [[status, log_id]]}]}

        if 'log_id' in fields:
            status = logger.update_log_record(fields) if logger else f"log not available"

        if 'query_filter' in fields:
            query_filter = fields.pop('query_filter')

        else:

            if not question and len(fields.keys()) == 1:
                question = payload['input_data'][0]['values'][0][0]
                fields = {}

        if not question:
            return {"predictions": [{"fields": ["status"], "values": [[status]]}]}

        # initialize time measurement list
        times = [('start', time.perf_counter())]

        try:

            if query_filter:

                inputs = {
                    "query": question,
                    "filter": [{"match": query_filter}]
                }
            else:
                inputs = {"query": question}

            llm_response = rag_chain_with_source.invoke(inputs)

            times.append(('llm_chain', time.perf_counter()))
            scoring_response_txt = {}
            scoring_response_txt['response'] = llm_response['answer']

            hallucination_response_text = "Sorry, I cannot find an answer to your question in the available documents."

            if llm_response['context']:
                llm_response_documents = [i['document'] for i in llm_response['context']]

                if parameters['default_hallucination_technique'] == "sentence_transformer":
                    hallucination_response = validate_answer_against_sources(llm_response['answer'], llm_response_documents, parameters["similarity_threshold_for_sentence_transformer_hallucination"])
                    hallucination_dict = {"Technique": "sentence_transformer", "confidence_score": hallucination_response['confidence_score']}
                elif parameters['default_hallucination_technique'] == "word_overlap":
                    hallucination_response = is_hallucination(llm_response['answer'], llm_response_documents, hallucination_threshold_max_text_overlap, hallucination_threshold_concatenated_text_overlap)
                    hallucination_dict = {"Technique": "Word Overlap", 
                                          "text_max_overlap_score": hallucination_response['maxOverlapScore'],
                                          "concatenated_text_overlap_score": hallucination_response['concatOverlapScore']}

                else:
                    hallucination_dict = {"Technique": "None"}
                    hallucination_response = {"isHallucination": False}
                if hallucination_response['isHallucination']:
                    scoring_response_txt['response'] = hallucination_response_text

                scoring_response_txt['source_documents'] = [{**response['document'].dict(), 'score': response['score']} for response in llm_response['context']]  # [doc for doc in llm_response['context']]

                scoring_response_txt['Hallucination Detection'] = hallucination_dict


            else:
                scoring_response_txt['response'] = hallucination_response_text
                scoring_response_txt['Hallucination Detection'] = { "Technique": parameters['default_hallucination_technique']}

            times.append(('hallucination_detection', time.perf_counter()))



        except Exception as e:

            print(f"Error while getting LLM response: {str(e)}")
            raise

        # post log record and return log id to caller
        if logging:
            scoring_response_txt['log_status'], scoring_response_txt['log_id'] = logger.create_log_record(question, scoring_response_txt, times, fields)
        

        scoring_response = {
            "predictions": [
                {
                    "fields": [
                        "llm_prediction"
                    ],
                    "values": [[scoring_response_txt]]
                }]}

        return scoring_response

    return score

Uncomment the below cells to test the scoring pipeline function locally in scope of the notebook. 

**Note:** To run the function locally you need to also uncomment all the subprocess calls to install the dependent libraries. Similarly, ensure that they are commented out before the function is deployed in the Deployment space to avoid library conflicts.  

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question"],"values": [["how to configure runtime environment in decision optimization?"]]}]})

We can filter the source documents title with a value `SQL optimization (SPSS Modeler)` and LLM will generate the response from only those source documents using the below cell.

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question", "query_filter"],"values": [["how to perform sql optimization?",{"metadata.title.keyword": {"query": "SQL optimization (SPSS Modeler)"}}]]}]})

<a id="DeployScoringFunction"></a>
### Deployment of function to Space

Initialize the function details, metadata and create the function deployment in the space

In [ ]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'rag_function_watsonx_ai_with_'+connection_type,
    client.repository.FunctionMetaNames.DESCRIPTION: 'QnA with RAG using' + connection_type,
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}

watsonx_function_details = client.repository.store_function(meta_props=meta_data, function=rag_scoring_pipeline_function)

watsonx_function_uid = client.repository.get_function_id(watsonx_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "rag_scoring_function_with_"+connection_type,
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "QnA with RAG using" + connection_type,
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'L'},  
   client.deployments.ConfigurationMetaNames.SERVING_NAME:  connection_type+"_rag_with_wx_" + str(uuid.uuid1()).split('-')[0]
}

watsonx_deployment_details = client.deployments.create(watsonx_function_uid, meta_props=meta_props)

watsonx_deployment_id = client.deployments.get_id(watsonx_deployment_details)

<a id="scoring"></a>
### Test the deployed scoring function

Test the deployed scoring function by passing in a question.

In [ ]:

payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["question"],
        "values": [["how to perform sql optimization?"]]
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

<a id="feedback-logging"></a>
### Feedback Logging

Updating feedback in log record. \
In case the answer returned by the LLM is relevant to your question provide the values below as postive along with a comment. Otherwise, provide the values below as negative. \
This feedback will be saved if an Elasticsearch index or Milvus collection is provided for feedback logging in the parameter set. 
In case of elasticsearch, you can view the indexed feedback logs via the elasticsearch **kibana** user interface under the **Index Management** section. \
Find the document for the log record that corresponds to the conversation above. It contains, among others, the question and response, documents found and the user's feedback.

In [ ]:
try:
    log_id = wx_result['predictions'][0]['values'][0][0]['log_id']
except NameError: 
    log_id = ''
    
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["log_id", "value", "comment"],
        "values": [[log_id, "positive", "Nice log record!"]]
       
    }]
}
wx_log_update_result = client.deployments.score(watsonx_deployment_id, payload)
wx_log_update_result

<a id="updateParameters"></a>
### Update parameter set in the project & deployment space

Update the advanced parameter set in both project & space with the deployment id of the function.

In [ ]:
paramset_name = "RAG_advanced_parameter_set"
parameter_to_be_updated = {"name":"wml_rag_deployment_id","value":watsonx_deployment_id}
rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated)

client.set.default_project(project_id=project_id)
if rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated) == True:
    print("Parameter set in the project and deployment space has been updated with the deployment id of the function above.")
else:
    print("Parameter set update failed.")

 Proceed to **Q&A with RAG** notebook to test out the deployed function by asking some questions.

**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**